In [1]:
import torch
import pandas as pd
from modules.MyTrainer import NILMTrainer
from modules.helpers import create_tree_dir, save_report, train_model, display_res,train_eval
from torch.utils.data import Dataset, DataLoader

from modules.MyDataSet import MyChunk, MyChunkList

with torch.no_grad():
    torch.cuda.empty_cache() 

Global seed set to 42


Device: cuda:0


# Project Settings

In [2]:
clean = True
ROOT = 'output'
data_dir = '/mnt/B40864F10864B450/WorkSpace/PHD/PHD_exps/data'
train_file_dir = 'dates/train/'
test_file_dir = 'dates/test/'

dev_list = ['fridge', 
#             'kettle', 
#             'washing machine',
#             'washer dryer',
#             'tumble dryer',
#             'dish washer',
#             'microwave',
#             'television',
#             'computer',
#             'electric space heater'
           ]
mod_list = [
#             'SF2P',
#             'S2P',
#             'SimpleGru',
#             'FFED',
#             'SAED',
#             'FNET',
#             'WGRU',
            'ConvFourier'
            ]
cat_list = [x for x in ['Single', 'Multi']]
tree_levels = {'root': ROOT, 'l1': ['results'], 'l2': dev_list, 'l3': mod_list, 'experiments':cat_list}
create_tree_dir(tree_levels=tree_levels, clean=clean)

root
all clean
/mnt/B40864F10864B450/WorkSpace/PHD/PHD_exps/NewPaper/torch_nilm/output
1


# Experiment settings

In [3]:
exp_type = 'Single'#'Multi'

EPOCHS = 5
ITERATIONS = 1

SAMPLE_PERIOD = 6
WINDOW = 50
device = 'fridge'
BATCH = 1024

model_hparams={
              'SimpleGru':{},
              'SAED':{'window_size':WINDOW},
              'FFED':{},
              'WGRU':{'dropout':0.25},
              'S2P': {'window_size':WINDOW, 'dropout':0.25},
              'ConvFourier': {'window_size':WINDOW, 'dropout':0.25},
              'SF2P': {'window_size':WINDOW, 'dropout':0.25},
              'FNET': {'depth': 2, 'kernel_size':5, 'cnn_dim': 64,
                       'input_dim':WINDOW, 'hidden_dim':WINDOW*8, 'dropout':0.25}
               }

In [4]:
test_houses = []
test_sets = []
test_dates = []
test_file = open('{}base{}TestSetsInfo_{}'.format(test_file_dir, exp_type, device), 'r')
for line in test_file:    
    toks = line.split(',')
    test_sets.append(toks[0])
    test_houses.append(toks[1])
    test_dates.append([str(toks[2]),str(toks[3].rstrip("\n"))])
test_file.close()
data = {'test_house': test_houses, 'test_set': test_sets, 'test_date': test_dates}
tests_params = pd.DataFrame(data)
tests_params

,test_house,test_set,test_date
0,1,UKDALE,"[2014-09-01, 2014-10-01]"
1,2,UKDALE,"[2013-05-22, 2013-06-22]"
2,4,UKDALE,"[2013-05-22, 2013-06-22]"
3,5,UKDALE,"[2014-08-01, 2014-09-01]"
4,5,REFIT,"[2014-09-01, 2014-10-01]"
5,6,REFIT,"[2014-09-01, 2014-10-01]"
6,20,REFIT,"[2015-01-01, 2015-02-01]"


# TrainLoader

In [5]:
train_file = open('{}base{}TrainSetsInfo_{}'.format(train_file_dir, exp_type, device), 'r')
if exp_type=='Single':
    for line in train_file:    
        toks = line.split(',')
        train_set = toks[0]
        train_house = toks[1]
        train_dates = [str(toks[2]),str(toks[3].rstrip("\n"))]
        break
    train_file.close()
    
    path = data_dir + '/{}/{}.h5'.format(train_set, train_set)
    print(path)
    train_dataset = MyChunk(path=path, building=int(train_house), window_size=WINDOW,
                      device=device, dates=train_dates, sample_period=SAMPLE_PERIOD)
else:
    for line in train_file:    
        toks = line.split(',')
        train_set = toks[0]
        break
    train_file.close()
    train_dataset = MyChunkList(device,filename=train_file,
                                window_size=WINDOW, sample_period=SAMPLE_PERIOD)
    
train_loader = DataLoader(train_dataset, batch_size=BATCH, 
                          shuffle=False, num_workers=8)
mmax = train_dataset.mmax

/mnt/B40864F10864B450/WorkSpace/PHD/PHD_exps/data/UKDALE/UKDALE.h5


/home/nick/anaconda3/envs/pytorch_nilm/lib/python3.8/site-packages/nilmtk/utils.py:502: UserWarning: Found duplicate index. Keeping first value
  warnings.warn("Found duplicate index. Keeping first value")
<ipython-input-5-57e6eaaa0720>:13: FutureWarning: Converting timezone-aware DatetimeArray to timezone-naive ndarray with 'datetime64[ns]' dtype. In the future, this will return an ndarray with 'object' dtype where each element is a 'pandas.Timestamp' with the correct 'tz'.
	To accept the future behavior, pass 'dtype=object'.
	To keep the old behavior, pass 'dtype="datetime64[ns]"'.
  train_dataset = MyChunk(path=path, building=int(train_house), window_size=WINDOW,


# Run Experiment

In [ ]:
experiments = []
experiment_name = '_'.join([device, exp_type,'Train',train_set,'',])
print(experiment_name)
eval_params={'device':device,
             'mmax':mmax,
             'groundtruth':''}
for model_name in mod_list:
    print('#'*40)
    print('MODEL: ', model_name)
    print('#'*40)
    for iteration in range(1,ITERATIONS+1):
        print('#'*20)
        print('Iteration: ', iteration)
        print('#'*20)
        experiment_name = '_'.join([device, exp_type,'Train',train_set,'',])
        experiments.append(experiment_name)
        train_eval(model_name, 
                   train_loader, 
                   exp_type, 
                   tests_params,
                   SAMPLE_PERIOD, 
                   BATCH, 
                   experiment_name, 
                   iteration,
                   device,
                   mmax,
                   WINDOW, 
                   ROOT, 
                   data_dir,
                   epochs=EPOCHS,
                   eval_params=eval_params,
                   model_hparams=model_hparams[model_name])

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


fridge_Single_Train_UKDALE_
########################################
MODEL:  ConvFourier
########################################
####################
Iteration:  1
####################



  | Name  | Type        | Params
--------------------------------------
0 | model | ConvFourier | 1.3 M 
--------------------------------------
1.3 M     Trainable params
0         Non-trainable params
1.3 M     Total params
5.132     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

/home/nick/anaconda3/envs/pytorch_nilm/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:52: UserWarning: The {log:dict keyword} was deprecated in 0.9.1 and will be removed in 1.0.0
Please use self.log(...) inside the lightningModule instead.
# log on a step or aggregate epoch metric to the logger and/or progress bar (inside LightningModule)
self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
  warnings.warn(*args, **kwargs)


In [ ]:
display_res(ROOT, model_name, device, exp_type,'fridge_Single_Train_UKDALE_test_1_UKDALE',
            1,low_lim=40000, upper_lim=50000)

In [ ]:
display_res(ROOT, model_name, device, exp_type,'fridge_Single_Train_UKDALE_test_2_UKDALE',
            1,low_lim=40000, upper_lim=50000)

In [ ]:
display_res(ROOT, model_name, device, exp_type,'fridge_Single_Train_UKDALE_test_4_UKDALE',
            1,low_lim=40000, upper_lim=50000)